In [48]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np 

In [93]:
def read(his_file,sim_num,fname): 

# Middle of the vegetation patch  
    jmid=50; imid=45; 
# detect the presence of vegetation patch to know its extents (Currently hardwired
    imin_plant=41 ; imax_plant=50 ; jmin_plant=45 ; jmax_plant=54   
    
    nc = netCDF4.Dataset(his_file)
    mask = nc.variables['mask_rho'][:]

# read x,y
    x = nc.variables['x_rho'][:,:]
    y = nc.variables['y_rho'][:,:] 
    tot_time = nc.variables['ocean_time'] 
    
    imax=size(x[1,:])
    jmax=size(y[:,1])
    
    # read water depth
    depth = nc.variables['h'][:,:]
    theta_s = nc.variables['theta_s'][:]
    theta_b = nc.variables['theta_b'][:]
    depth_c = nc.variables['hc'][:]
    
#Get Stretching function 
    N=10 # Number of vertical levels, kgrid=1 MATLAB code copy 
    Np=N+1 
    ds=1.0/N
    s=np.zeros(11)
    Csur=np.zeros(11)
    for i in range(0,Np):
        lev=i
        s[i]=(lev-N)*ds
        Csur[i]=-s[i]**2
    
# Reshape 1D vertical variables to 3D so we can broadcast
    s.shape=(np.size(s),1,1)
    Csur.shape=(np.size(Csur),1,1)
    
# read 3D water level at all time steps
    eta = nc.variables['zeta'] 
    
# calculate the 4D field of z values (vertical coordinate)
    z0=(depth_c*s+depth*Csur)/(depth_c+depth)
    
# Extract the time frame for the last tidal cycle
    t_tidal=[]
    for t in range(1,size(tot_time)):
        if(eta[t,jmid,imid]<0.0 and eta[t+1,jmid,imid]>0.0):
            t_tidal.append(t)

    t_tidal_srt=t_tidal[-2]
    t_tidal_end=t_tidal[-1]
    print "Last tidal cycle starts and ends at this instance" 
    print t_tidal_srt,t_tidal_end   
    t_slice=t_tidal_end-t_tidal_srt+1
       
    bef_j=jmin_plant-3
    aft_j=jmax_plant+3
     
    ke_bef=0.0 
    ke_aft=0.0              
    # Calculate change in kinetic energy before and after the patch
    for t in range (t_tidal_srt,t_tidal_end+1): 
        u=nc.variables['u'][t,:,:,:]
        v=nc.variables['v'][t,:,:,:]
        w=nc.variables['w'][t,:,:,:]
    
        for k in range(1,N):
            for i in range(imin_plant-3,imax_plant+4):
                u_c=0.5*(u[k,bef_j,i]   + u[k,bef_j,i+1])
                v_c=0.5*(v[k,bef_j+1,i] + v[k,bef_j,i])
                w_c=0.5*(w[k,bef_j,i]   + w[k-1,bef_j,i])
                ke_bef=sqrt(u_c*u_c+v_c*v_c+w_c*w_c)+ke_bef
            
                u_c=0.5*(u[k,aft_j,i]   + u[k,aft_j,i+1])
                v_c=0.5*(v[k,aft_j+1,i] + v[k,aft_j,i])
                w_c=0.5*(w[k,aft_j,i]   + w[k-1,aft_j,i])
                ke_aft=sqrt(u_c*u_c+v_c*v_c+w_c*w_c)+ke_aft
        
    mean_delta_ke=(ke_aft-ke_bef)/t_slice
    
    print "mean_delta_ke"
    print mean_delta_ke 
    
    f.write("%s\t, %s\t \n" % (sim_num, mean_delta_ke))


In [94]:
'''Output from vegetation module COAWST 
Mean chane in planar kinetic energy (before & after the around vegetation patch [(Watt/m2 * m3)/time = Watt-mts/second]
'''

fname='ke_change_output.txt'
f = open(fname, 'w')
total_sim_list=16
f.write("Case_no, Mean KE change(mts2/sec3): \n")

for sim_num in range(1,total_sim_list):
    path1='/media/gadar/DATADRIVE1/sensitivity_history/'
    path2='veg_test_his_'+ str(sim_num)+'.nc'
    url=path1+path2
    print url 
    read(url,sim_num,fname) 
    
f.close()

/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_1.nc
Last tidal cycle starts and ends at this instance
186 258
mean_delta_ke
9.28511113565
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_2.nc
Last tidal cycle starts and ends at this instance
186 258
mean_delta_ke
8.59116120376
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_3.nc
Last tidal cycle starts and ends at this instance
186 258
mean_delta_ke
9.20725286751
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_4.nc
Last tidal cycle starts and ends at this instance
186 258
mean_delta_ke
9.22159875619
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_5.nc
Last tidal cycle starts and ends at this instance
186 258
mean_delta_ke
9.19565064461
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_6.nc
Last tidal cycle starts and ends at this instance
186 258
mean_delta_ke
8.51969919061
/media/gadar/DATADRIVE1/sensitivity_history/veg_test_his_7.nc
Last tidal cycle starts and ends at this instance
